In [68]:
# things we need for NLP
import joblib
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [69]:
# things we need for Tensorflow
import numpy as np
import matplotlib.pyplot as plt
import tflearn
import tensorflow as tf
import random
import pandas as pd
import re
import string
from nltk.stem import WordNetLemmatizer
from collections import Counter
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [70]:
# import our chat-bot intents file
import json
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

In [71]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

71 documents
14 classes ['Genre', 'goodbye', 'languages_ar', 'languages_cn', 'languages_de', 'languages_en', 'languages_es', 'languages_fr', 'languages_hi', 'languages_it', 'languages_ja', 'languages_kr', 'languages_pt', 'languages_ru']
41 unique stemmed words ['ar', 'arab', 'as', 'bye', 'categ', 'chin', 'chines', 'engl', 'film', 'frant', 'french', 'genr', 'germ', 'germany', 'goodby', 'hind', 'in', 'ind', 'it', 'ita', 'jap', 'japanes', 'kor', 'lang', 'langu', 'lat', 'latin', 'latino', 'list', 'movy', 'of', 'portugues', 'russ', 'see', 'slav', 'sort', 'span', 'ther', 'typ', 'what', 'you']


In [72]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-72-9d149b3413ff>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [73]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
history = model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Training Step: 8999  | total loss: 0.88332 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.88332 - acc: 0.6486 -- iter: 64/71
Training Step: 9000  | total loss: 0.97105 | time: 0.033s
| Adam | epoch: 1000 | loss: 0.97105 - acc: 0.6266 -- iter: 71/71
--
INFO:tensorflow:C:\Users\Daniel Krasovski\Documents\GitHub\Project_Oreo\models\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [74]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "models/training_data", "wb" ) )

